<a href="https://colab.research.google.com/github/GuyRobot/AINotesBook/blob/main/TextStoriesGenerate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 1.x


TensorFlow 1.x selected.


In [ ]:

import os
import tensorflow as tf
import numpy as np

import collections
from six.moves.urllib.request import urlretrieve
import time

tf.compat.v1.enable_eager_execution()


In [ ]:

url = 'https://www.cs.cmu.edu/~spok/grimmtmp/'

dir_name = 'stories'
if not os.path.exists(dir_name):
    os.mkdir(dir_name)

def download_stories(filename):
    print("Downloading file: ", dir_name + os.sep + filename)
    if not os.path.exists(os.path.join(dir_name, filename)):
        filename, _ = urlretrieve(url + filename, dir_name+os.sep+filename)
    else:
        print("File %s already exits" % filename)
    return filename


filenames = [format(i, '03d') + '.txt' for i in range(1, 101)]

for fn in filenames:
    download_stories(fn)


In [ ]:

filenames = [format(i, '03d') + '.txt' for i in range(1, 101)]
dir_name = 'stories'

def read_data(filename):
    with open(filename) as f:
        data =  tf.compat.as_str(f.read())
        data = data.lower()
        data = list(data)
    return data

global documents
documents = []
num_files = 100
for i in range(num_files):
    print("processing file %s" % os.path.join(dir_name, filenames[i]))
    chars = read_data(os.path.join(dir_name, filenames[i]))

    # break into bigrams
    two_grams = [''.join(chars[ch_i:ch_i+2]) for ch_i in range(0, len(chars)-2, 2)]
    # Create document
    documents.append(two_grams)
    print("Data size (chars) (document %d) %d" % (i, len(two_grams)))
    print("Sample string %s\n" % (two_grams[:50]))


processing file stories/001.txt
Data size (chars) (document 0) 3667
Sample string ['in', ' o', 'ld', 'en', ' t', 'im', 'es', ' w', 'he', 'n ', 'wi', 'sh', 'in', 'g ', 'st', 'il', 'l ', 'he', 'lp', 'ed', ' o', 'ne', ', ', 'th', 'er', 'e ', 'li', 've', 'd ', 'a ', 'ki', 'ng', '\nw', 'ho', 'se', ' d', 'au', 'gh', 'te', 'rs', ' w', 'er', 'e ', 'al', 'l ', 'be', 'au', 'ti', 'fu', 'l,']

processing file stories/002.txt
Data size (chars) (document 1) 4928
Sample string ['ha', 'rd', ' b', 'y ', 'a ', 'gr', 'ea', 't ', 'fo', 're', 'st', ' d', 'we', 'lt', ' a', ' w', 'oo', 'd-', 'cu', 'tt', 'er', ' w', 'it', 'h ', 'hi', 's ', 'wi', 'fe', ', ', 'wh', 'o ', 'ha', 'd ', 'an', '\no', 'nl', 'y ', 'ch', 'il', 'd,', ' a', ' l', 'it', 'tl', 'e ', 'gi', 'rl', ' t', 'hr', 'ee']

processing file stories/003.txt
Data size (chars) (document 2) 9745
Sample string ['a ', 'ce', 'rt', 'ai', 'n ', 'fa', 'th', 'er', ' h', 'ad', ' t', 'wo', ' s', 'on', 's,', ' t', 'he', ' e', 'ld', 'er', ' o', 'f ', 'wh', 'om', ' w

In [ ]:

def build_dataset(documents):
    chars = []
    # list of lists
    data_list = []

    for d in documents:
        chars.extend(d)
    print('%d character found.' % len(chars))

    count = []
    # bigrams sorted by their frequency
    count.extend(collections.Counter(chars).most_common())

    # Create dict map word to id by given the current length of the dictionary
    # UNK is for two rare word
    dictionary = dict({'UNK': 0})
    for char, c in count:
        # Only add if its frequency is more than 10
        if c > 10:
            dictionary[char] = len(dictionary)
    unk_count = 0
    # replace word with id of word
    for d in documents:
        data = list()
        for char in d:
            # if word in dictionary use the id of word
            # otherwise use id of UNK
            if char in dictionary:
                index = dictionary[char]
            else:
                index = dictionary['UNK']
                unk_count += 1
            data.append(index)
        data_list.append(data)

    # dict map id to word
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data_list, count, dictionary, reverse_dictionary

data_list, count, dictionary, reverse_dictionary = build_dataset(documents)
print('Most common words (+UNK)', count[:5])
print('Least common words (+UNK)', count[-15:])
print('Sample data', data_list[0][:10])
print('Sample data', data_list[1][:10])
print('Vocabulary: ',len(dictionary))
vocabulary_size = len(dictionary)
del documents  # To reduce memory.



449177 character found.
Most common words (+UNK) [('e ', 15229), ('he', 15164), (' t', 13443), ('th', 13076), ('d ', 10687)]
Least common words (+UNK) [('bj', 1), ('ii', 1), ('i?', 1), ('z ', 1), ('c.', 1), ('"k', 1), ('pw', 1), ('f?', 1), (' z', 1), ('xq', 1), ('nm', 1), ('m?', 1), ('\t"', 1), ('\tw', 1), ('tz', 1)]
Sample data [15, 28, 86, 23, 3, 95, 74, 11, 2, 16]
Sample data [22, 156, 25, 37, 82, 185, 43, 9, 90, 19]
Vocabulary:  544


In [ ]:

all_data = [data for i in data_list for data in i]
all_data = tf.convert_to_tensor(all_data, dtype=tf.int64)
len(all_data)



449177

In [ ]:

# datasets = [tf.data.Dataset.from_tensor_slices(data) for data in data_list]
# datasets
dataset = tf.data.Dataset.from_tensor_slices(all_data, )
dataset


<DatasetV1Adapter shapes: (), types: tf.int64>

In [ ]:

for i in dataset.take(5):
    print(reverse_dictionary[i.numpy()])


in
 o
ld
en
 t


In [ ]:


seq_length = 64
examples_per_epoch = len(all_data) // (seq_length + 1)

dataset_sequences  = dataset.batch(seq_length + 1, drop_remainder=True)

for item in dataset_sequences.take(5):
    for c in item.numpy():
        print(reverse_dictionary[c], end='')
    print("\n" + "-" * 24)



in olden times when wishing still helped one, there lived a king
whose daughters were all beautiful, but the youngest was so beaut
------------------------
iful
that the sun itself, which has seen so much, was astonished whenever
it shone in her face.  close by the king's castle lay a 
------------------------
great dark
forest, and under an old lime-tree in the forest was a well, and when
the day was very warm, the king's child went out 
------------------------
into the forest and
sat down by the side of the cool fountain, and when she was bored she
took a golden ball, and threw it up on h
------------------------
igh and caught it, and this
ball was her favorite plaything.

now it so happened that on one occasion the princess's golden ball
d
------------------------


In [ ]:

# Map text to input and target (both input and target have
# the same seq_length but target is shifted to right one character)
def split_input_target(chunk):
    input_text = chunk[:-1] # take all except the last character
    target_text = chunk[1:] # take all except the first character

    return input_text, target_text

dataset = dataset_sequences.map(split_input_target)
dataset



<DatasetV1Adapter shapes: ((64,), (64,)), types: (tf.int64, tf.int64)>

In [ ]:

for input_exp, target_exp in dataset.take(2):
    print("\nInput data: ")
    for i in input_exp.numpy():
        print(reverse_dictionary[i], end='')
    print("\nTarget data: ")
    for t in target_exp.numpy():
        print(reverse_dictionary[t], end='')



Input data: 
in olden times when wishing still helped one, there lived a king
whose daughters were all beautiful, but the youngest was so bea
Target data: 
 olden times when wishing still helped one, there lived a king
whose daughters were all beautiful, but the youngest was so beaut
Input data: 
iful
that the sun itself, which has seen so much, was astonished whenever
it shone in her face.  close by the king's castle lay 
Target data: 
ul
that the sun itself, which has seen so much, was astonished whenever
it shone in her face.  close by the king's castle lay a 

In [ ]:

BATCH_SIZE = 32
embedding_dim = 64
steps_per_epoch = examples_per_epoch//BATCH_SIZE

rnn_units = 1024


dataset = dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(1)

dataset


<DatasetV1Adapter shapes: ((32, 64), (32, 64)), types: (tf.int64, tf.int64)>

In [ ]:
for i in dataset.take(1):
    print(i)

(<tf.Tensor: id=67, shape=(32, 64), dtype=int64, numpy=
array([[ 15,  28,  86, ..., 100,  25,  43],
       [142, 123,  99, ...,   1, 138,  37],
       [185,  43,   9, ...,  23,   9,  21],
       ...,
       [ 16, 229,  44, ...,  80, 204,  58],
       [ 29, 102,   2, ...,   7,  10,   5],
       [ 55,  17,  27, ...,  36,  42,  40]])>, <tf.Tensor: id=68, shape=(32, 64), dtype=int64, numpy=
array([[ 28,  86,  23, ...,  25,  43,  60],
       [123,  99,  22, ..., 138,  37,  82],
       [ 43,   9, 199, ...,   9,  21,   9],
       ...,
       [229,  44,  36, ..., 204,  58,   5],
       [102,   2,  12, ...,  10,   5,  46],
       [ 17,  27,  38, ...,  42,  40,  87]])>)


In [ ]:


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=(batch_size, None)),
        tf.keras.layers.GRU(units=rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model


model = build_model(vocabulary_size, embedding_dim, rnn_units, batch_size=BATCH_SIZE)
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (32, None, 64)            34816     
_________________________________________________________________
gru (GRU)                    (32, None, 1024)          3345408   
_________________________________________________________________
dense (Dense)                (32, None, 544)           557600    
Total params: 3,937,824
Trainable params: 3,937,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:

"""
    For each character the model looks up the
    embedding, runs the GRU one timestep with
    the embedding as input, and applies the dense
    layer to generate logits predicting the log-likelihood of the next character:
"""
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

# my_model.summary()



(32, 64, 544) # (batch_size, sequence_length, vocab_size)


In [ ]:

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()
sampled_indices


array([471, 227, 434, 432, 147, 181, 186, 540,  20, 385, 161, 294, 300,
       302, 226,   6,   8,  64, 145, 450, 267,  83,  27, 150,  79, 333,
         1, 203, 214, 429, 497,  21,  35, 432, 476, 149, 306, 157, 237,
       509, 537, 536, 500, 373,  14,  15, 255, 492, 121, 367, 153, 379,
       540, 282, 230, 269, 228, 437, 443, 328,  80,  84, 208, 438])

In [ ]:

def loss_sparse(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss_sparse(target_example_batch, example_batch_predictions)
example_batch_loss.numpy().mean()


6.299143

In [ ]:

optimizer = tf.train.AdamOptimizer()

def train_step(inp, target):
    with tf.GradientTape() as g:
        predictions = model(inp)
        loss = tf.reduce_mean(loss_sparse(target, predictions))

    gradients = g.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    return loss


In [ ]:

# EPOCHS = 10
# checkpoint_dir = './stories_checkpoint'

# checkpoint_predix = os.path.join(checkpoint_dir, 'ckpt_{epoch}')

# checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_predix,
#     save_weights_only=True
# )

# for epoch in range(EPOCHS):
#     start = time.time()
#     loss = 0
#     # reset hidden state
#     model.reset_states()

#     for (batch_n, (inp, target)) in enumerate(dataset):
#         loss = train_step(inp, target)

#         if batch_n % 100 == 0:
#             print('Epoch {} Batch {} Loss {}'.format(epoch + 1, batch_n, loss))

#     if (epoch + 1) % 5 == 0:
#         model.save_weights(checkpoint_predix.format(epoch=epoch))

#     print('Epoch {} Loss {:.4f}'.format(epoch + 1, loss))
#     print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))


# model.save_weights(checkpoint_predix.format(epoch=epoch))



In [ ]:

model.compile(
    optimizer = tf.train.AdamOptimizer(),
    loss = loss_sparse)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './stories_test_checkpoint'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:

EPOCHS = 64
history = model.fit(dataset.repeat(), epochs=EPOCHS, steps_per_epoch=steps_per_epoch, callbacks=[checkpoint_callback])



Epoch 1/64
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
215/215 [==============================] - 30s 139ms/step - loss: 4.4428
Epoch 2/64
215/215 [==============================] - 29s 134ms/step - loss: 3.2972
Epoch 3/64
215/215 [==============================] - 28s 132ms/step - loss: 2.9668
Epoch 4/64
215/215 [==============================] - 28s 132ms/step - loss: 2.7475
Epoch 5/64
215/215 [==============================] - 29s 133ms/step - loss: 2.5655
Epoch 6/64
215/215 [==============================] - 28s 130ms/step - loss: 2.3906
Epoch 7/64
215/215 [==============================] - 28s 132ms/step - loss: 2.2062
Epoch 8/64
215/215 [==============================] - 29s 135ms/step - loss: 2.0235
Epoch 9/64
215/215 [==============================] - 29s 133ms/step - loss: 1.8838
Epoch 10/64
215/215 [==============================] - 28s 131ms/step - loss: 1.7673
Epoch 11/64
215/215 [==============================] - 29s 133ms/

In [ ]:

model = build_model(vocabulary_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 64)             34816     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3345408   
_________________________________________________________________
dense_1 (Dense)              (1, None, 544)            557600    
Total params: 3,937,824
Trainable params: 3,937,824
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Vectorize
char2idx = {u:i for i, u in zip(dictionary.values(), dictionary.keys())}
idx2char = {i:u for i, u in zip(dictionary.values(), dictionary.keys())}

# Convert all character to int base on char2idx dict
text_as_int = np.array(all_data)
len(idx2char)

544

In [ ]:

def generate_text_bigram(model, start_string):
  # Evaluation step (generating text using the learned model)
  # Number of characters to generate
  num_generate = 1000
  two_grams = [''.join(start_string[ch_i:ch_i+2]) for ch_i in range(0, len(start_string)-2, 2)]
  two_grams_ids = []
  for i in two_grams:
    two_grams_ids.append(dictionary[i])
  # Converting our start string to numbers (vectorizing)
  input_eval = tf.convert_to_tensor(two_grams_ids)
  input_eval = tf.expand_dims(input_eval, 0)
  print(input_eval.shape)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
      predictions = tf.squeeze(predictions, 0)

      # using a multinomial distribution to predict the word returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted word as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))



In [ ]:
print(generate_text_bigram(model, start_string=u"a certain father had two sons, the elder of whom was smart ands"))


(1, 31)
a certain father had two sons, the elder of whom was smart ands attendants
which
to into
the king's you.  i will
travely
you, and if i small not be able to give the
middle me the
tailor died, "an you sat
my good horbs into the right, it would have eaten
hy and his time, and believed that,
after all, she had only for a keep which they had thought
to himself,
"thus it came in.  the king
said to him, "you a
come.  i have been asleep." but it?" had a could who he grows sleeps.  the bride bed, jemake them
go with
them and put a
bit off oy
out of his sput, that there was no bringing it out,
with great fellowed," and will give him in a threat and
wision to him.  you are," said they, and said, "open that tere but
faithful to the king's come, they lived on your beloath with
the gole."  when the old kinger said,
"when darkness cannot ring-maider.  when they were full of may were alive, when i am a ping.

she said
up on any had apprentice in the first ston she lay down until her human tog

In [ ]:
from google.colab import files
files.download('stories_test_checkpoint/ckpt_64.data-00001-of-00002') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import pickle
with open('stories_dictionary.pickle', 'wb') as handle:
    pickle.dump(dictionary, handle, protocol=pickle.HIGHEST_PROTOCOL)